In [16]:
import datetime
import itertools
import json
import numpy as np
import os
import pandas as pd
from pandas_helpers import *
import pickle
import pprint
import random
import re
import requests
import seaborn
import sys
import time

pd.set_option('display.precision', 20)

DATA_DIR = 'data/citydata/transit'

random.seed(90210)

# Transit Data

In [5]:
!cat $DATA_DIR/Late_night_ridership_data_README.txt

MBTA LATE NIGHT DATA

Data is number of transactions recorded by MBTA fare gates, buses, and light rail vehicles
Only includes transactions after 10pm on Fridays and Saturdays from March 1 2013 to June 14 2014
Transactions grouped by date, route or line, and 15-minute window.

Field definitions

scheduledate: service date, which includes hours after midnight (e.g. 1am on June 14 has a service date of June 13)
latenightroute: 0 = no late night service scheduled, 1 = late night service scheduled.
	This only indicates whether a route currently has late night service, not whether it had late night on that particular day
	Intended to facilitate comparisons for routes whose hours were extended
	Some routes will still have transactions after 1am even though they do not have extended hours, this is because their last trip is around 1am
line: indicates rapid transit line (Red, Blue, Orange, Green, Silver) or bus
routestation: indicates the station for rapid transit services and the route for bu

In [230]:
df = pd.read_csv(os.path.join(DATA_DIR, 'LateNight_thru_7June2014.csv'), 
                 parse_dates=['scheduledate'],
                 usecols=['scheduledate', 'trxhour', 'line', 'transactions'])

# There are roughly 3k rows with nothing in them at the bottom of the csv
df.dropna(inplace=True)

In [231]:
df.head()

,scheduledate,line,trxhour,transactions
0,2013-03-01,Bus,22,17
1,2013-03-01,Bus,22,8
2,2013-03-01,Bus,22,8
3,2013-03-01,Bus,22,5
4,2013-03-01,Bus,23,4


In [232]:
num_transactions = df['transactions'].sum()

print '{:d} rows'.format(len(df))
print '{:d} transactions'.format(int(num_transactions))

238477 rows
6768658 transactions


In [233]:
df = df.groupby(['scheduledate', 'trxhour', 'line']).sum().reset_index()
df.head()

,scheduledate,trxhour,line,transactions
0,2013-03-01,22,Blue,963
1,2013-03-01,22,Bus,7016
2,2013-03-01,22,Green,6930
3,2013-03-01,22,Orange,6928
4,2013-03-01,22,Red,5382


In [234]:
assert df['transactions'].sum() == num_transactions

In [235]:
def actual_date(x):
    # normalize date and hour
    if x['trxhour'] >= 24:
        x['scheduledate'] = x['scheduledate'] + datetime.timedelta(days=1)
        x['trxhour'] = x['trxhour'] - 24
    
    # set scheduledate to include the hour count
    x['scheduledate'] = x['scheduledate'] + datetime.timedelta(hours=x['trxhour'])
        
    return x

In [236]:
df = df.apply(actual_date, axis=1)

In [237]:
df.head()

,scheduledate,trxhour,line,transactions
0,2013-03-01 22:00:00,22,Blue,963
1,2013-03-01 22:00:00,22,Bus,7016
2,2013-03-01 22:00:00,22,Green,6930
3,2013-03-01 22:00:00,22,Orange,6928
4,2013-03-01 22:00:00,22,Red,5382


In [238]:
assert df['transactions'].sum() == num_transactions

In [239]:
df = pd.pivot_table(df, values='transactions', index='scheduledate', columns='line', aggfunc=np.sum)
df.head()

line,Blue,Bus,Green,Orange,Red,Silver
scheduledate,,,,,,
2013-03-01 22:00:00,963,7016,6930,6928,5382,357
2013-03-01 23:00:00,708,5359,4668,3371,4382,289
2013-03-02 00:00:00,331,2965,2625,1531,1823,108
2013-03-02 01:00:00,25,762,108,135,58,10
2013-03-02 02:00:00,1,99,2,6,6,NaN


In [240]:
df.fillna(0, axis=1, inplace=True)
df.head()

line,Blue,Bus,Green,Orange,Red,Silver
scheduledate,,,,,,
2013-03-01 22:00:00,963,7016,6930,6928,5382,357
2013-03-01 23:00:00,708,5359,4668,3371,4382,289
2013-03-02 00:00:00,331,2965,2625,1531,1823,108
2013-03-02 01:00:00,25,762,108,135,58,10
2013-03-02 02:00:00,1,99,2,6,6,0


In [241]:
df = df.reset_index().sort('scheduledate')
df.head()

line,scheduledate,Blue,Bus,Green,Orange,Red,Silver
0,2013-03-01 22:00:00,963,7016,6930,6928,5382,357
1,2013-03-01 23:00:00,708,5359,4668,3371,4382,289
2,2013-03-02 00:00:00,331,2965,2625,1531,1823,108
3,2013-03-02 01:00:00,25,762,108,135,58,10
4,2013-03-02 02:00:00,1,99,2,6,6,0


In [242]:
def export_normalize(row):
    row['date'] = '{d.year}-{d.month}-{d.day}'.format(d=row['scheduledate'])
    row['start_date'] = int((row['scheduledate'] - datetime.datetime(1970, 1, 1)).total_seconds() * 1000)
    
    return row

df = df.apply(export_normalize, axis=1)
df.head()

line,scheduledate,Blue,Bus,Green,Orange,Red,Silver,date,start_date
0,2013-03-01 22:00:00,963,7016,6930,6928,5382,357,2013-3-1,1362175200000
1,2013-03-01 23:00:00,708,5359,4668,3371,4382,289,2013-3-1,1362178800000
2,2013-03-02 00:00:00,331,2965,2625,1531,1823,108,2013-3-2,1362182400000
3,2013-03-02 01:00:00,25,762,108,135,58,10,2013-3-2,1362186000000
4,2013-03-02 02:00:00,1,99,2,6,6,0,2013-3-2,1362189600000


In [243]:
try:
    df.drop('scheduledate', axis=1, inplace=True)
    df.drop('index', axis=1, inplace=True)
except ValueError:
    pass

In [246]:
df.head()

line,Blue,Bus,Green,Orange,Red,Silver,date,start_date
0,963,7016,6930,6928,5382,357,2013-3-1,1362175200000
1,708,5359,4668,3371,4382,289,2013-3-1,1362178800000
2,331,2965,2625,1531,1823,108,2013-3-2,1362182400000
3,25,762,108,135,58,10,2013-3-2,1362186000000
4,1,99,2,6,6,0,2013-3-2,1362189600000


In [247]:
df['total'] = df[['Blue', 'Bus', 'Green', 'Orange', 'Red', 'Silver']].sum(axis=1)

In [248]:
df.head()

line,Blue,Bus,Green,Orange,Red,Silver,date,start_date,total
0,963,7016,6930,6928,5382,357,2013-3-1,1362175200000,27576
1,708,5359,4668,3371,4382,289,2013-3-1,1362178800000,18777
2,331,2965,2625,1531,1823,108,2013-3-2,1362182400000,9383
3,25,762,108,135,58,10,2013-3-2,1362186000000,1098
4,1,99,2,6,6,0,2013-3-2,1362189600000,114


In [249]:
assert df['total'].sum() == num_transactions

In [251]:
df = df_rearrange_columns(df, 
                          ['blue', 'bus', 'green', 'orange', 'red', 'silver', 'date', 'start_date', 'total'],
                          ['date', 'start_date', 'bus', 'blue', 'green', 'orange', 'red', 'silver', 'total'])   

In [252]:
df.head()

,date,start_date,bus,blue,green,orange,red,silver,total
0,2013-3-1,1362175200000,7016,963,6930,6928,5382,357,27576
1,2013-3-1,1362178800000,5359,708,4668,3371,4382,289,18777
2,2013-3-2,1362182400000,2965,331,2625,1531,1823,108,9383
3,2013-3-2,1362186000000,762,25,108,135,58,10,1098
4,2013-3-2,1362189600000,99,1,2,6,6,0,114


In [254]:
df[['bus', 'blue', 'green', 'orange', 'red', 'silver', 'total']] = df[['bus', 'blue', 'green', 'orange', 'red', 'silver', 'total']].astype(int)

In [255]:
df_to_json_split_wo_index(df, 
                          'transit.json', 
                          ['Day of Event',
                           'Hour of Event (ms from epoch)',
                           'Number of Bus Events',
                           'Number of Blue Line Rail Events',
                           'Number of Green Line Rail Events',
                           'Number of Orange Line Rail Events',
                           'Number of Red Line Rail Events',
                           'Number of Silver Line Rail Events',
                           'Number of Total Transit Events'])

In [256]:
# Sanity Checks
with open('transit.json', 'rb') as infile:
    data = json.load(infile)
    print sum([x[-1] for x in data['data']])

6768658


In [258]:
len(df), len(pd.unique(df.start_date))

(670, 670)